In [ ]:
import os
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt

MIN_WAVELENGTH = 200
MAX_WAVELENGTH = 760

exp_code = "perm_v2"

# specify path to raw data here
GLOBAL_PATH = r"M:\\0-UVVis\\RR"
PROJECT_PATH = os.path.join(GLOBAL_PATH, exp_code)
REF = os.path.join(GLOBAL_PATH, "water", "20250508_153708_484601_water.csv")

DELIMITER = ","
df_blank = pd.read_csv(
    REF, names=["wavelength", "transmittance"], sep=DELIMITER, skiprows=6
)
df_blank = df_blank[
    df_blank["wavelength"].between(MIN_WAVELENGTH, MAX_WAVELENGTH, inclusive="both")
]


def lambert_beer(
    intensity_blank: list[float], intensity_sample: list[float]
) -> np.ndarray:
    """A = log(T_0 / T_t)"""
    return np.log10(intensity_blank / intensity_sample)

In [ ]:
plt.rcParams.update(
    {
        "font.size": 24,
        "axes.linewidth": 3,
        "xtick.major.width": 3,
        "ytick.major.width": 3,
        "xtick.minor.width": 3,
        "ytick.minor.width": 3,
    }
)

In [ ]:
from datetime import datetime
import re
from tqdm import tqdm


data = []
raw_data = []  # for debugging
times = []
starttime = None

file_names = [
    os.path.join(PROJECT_PATH, file_name)
    for file_name in os.listdir(PROJECT_PATH)
    if os.path.isfile(os.path.join(PROJECT_PATH, file_name))
    and file_name.endswith(".csv")
]

for file_name in tqdm(file_names):
    # for second precision, use '%Y%m%d_%H%M%S'
    timestamp = datetime.strptime(
        re.search(r"(\d{8}_\d{6})", file_name).group(1), "%Y%m%d_%H%M%S"
    )
    if not starttime:
        starttime = timestamp
    times.append((timestamp - starttime).total_seconds() / 60)

    df_curr_spectrum = pd.read_csv(
        file_name,
        names=["wavelength", "transmittance"],
        delimiter=DELIMITER,
        skiprows=6,
    )
    df_curr_spectrum = df_curr_spectrum[
        df_curr_spectrum["wavelength"].between(
            MIN_WAVELENGTH, MAX_WAVELENGTH, inclusive="both"
        )
    ]
    assert len(df_blank) == len(df_curr_spectrum), file_name

    data.append(
        lambert_beer(df_blank["transmittance"], df_curr_spectrum["transmittance"])
    )
    raw_data.append(df_curr_spectrum["transmittance"])

In [ ]:
from scipy.stats import linregress


def detect_plateau_from_slope(
    values, num_datapoints: int = 5, threshold: float = 1e-4
) -> int:
    """
    Detects a plateau in a list of values by calculating the slope of a linear regression
    for the last num_datapoints and comparing it to a threshold value.
    """
    for i in range(len(values) - num_datapoints):
        if i < num_datapoints:
            continue
        slope = linregress(
            range(num_datapoints),
            np.array(values[i - num_datapoints : i]) / max(values),
        ).slope
        if abs(slope) < threshold:
            print(f"Reaction has reached plateau after {i} datapoints.")
            return i
    return len(values)

In [ ]:
common_x_axis = df_blank["wavelength"]
spectra_array = data[20:]
times = times[20:]

In [ ]:
from scipy.integrate import trapezoid
from tqdm import tqdm

error_tolerance = 0.1


# Calculate Jaccard index for pair of spectra
def jaccard_next_two_spectra(index_1, index_2, reference_spectrum=None):
    if reference_spectrum is None:
        # spectrum1_y = spectra[timestamps[index_1]].y
        spectrum1_y = spectra_array[index_1]
    else:
        spectrum1_y = reference_spectrum
    # spectrum2_y = spectra[timestamps[index_2]].y
    spectrum2_y = spectra_array[index_2]
    union, intersection = [], []
    for y1, y2 in zip(spectrum1_y, spectrum2_y):
        if abs(y1 - y2) < error_tolerance:
            union.append(y2)
            intersection.append(y2)
        else:
            union.append(max(y1, y2))
            intersection.append(min(y1, y2))
    intersection_area = trapezoid(intersection, common_x_axis)
    union_area = trapezoid(union, common_x_axis)
    return intersection_area / union_area


print("Calculating Jaccard indices...")
jaccards = []
for i in tqdm(range(len(spectra_array))):
    jaccards.append(jaccard_next_two_spectra(0, i))

print("Calculating reverse Jaccard indices...")
reverse_jaccards = []
for i in tqdm(range(len(spectra_array))):
    reverse_jaccards.append(jaccard_next_two_spectra(i, len(spectra_array) - 1))

print("Calclating neighbor Jaccard indices...")
neighbor_jaccards = []
for i in tqdm(range(len(spectra_array) - 1)):
    neighbor_jaccards.append(jaccard_next_two_spectra(i, i + 1))

In [ ]:
plateau_index = detect_plateau_from_slope(jaccards, num_datapoints=20, threshold=1e-4)

plt.figure(figsize=(8, 6))

plt.plot(
    times, jaccards, marker="o", label=f"Jaccard index", linestyle="None", markersize=3
)
# plot vertical line at plateau index
plt.axvline(
    x=times[plateau_index],
    color="r",
    linestyle="--",
    label=f"Plateau after {times[plateau_index]:.0f} min",
)

plt.legend(loc="upper right").get_frame().set_linewidth(3)
plt.xlabel("Time / min")
plt.ylabel("Jaccard index")
# plt.title(f'Jaccard index for {exp_code}')
plt.xlim(0, 50)
plt.savefig(
    os.path.join(f"{exp_code}_jaccard.svg"), transparent=True, bbox_inches="tight"
)